In [1]:
import pandas as pd
import os 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from nltk.corpus import stopwords
from nltk import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import string
import matplotlib.pyplot as plt;

import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
import nltk
#for first time using nltk
# nltk.download ()

In [2]:
text_variable='clean_text'
target_feature='category'
splits=10
stop_words = stopwords.words('english') 
dataset_location="datasets/Twitter and Reddit Sentimental analysis Dataset/Twitter_Data.csv"

In [3]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"(http\S+|http)", "", txt) # remove links 
    txt = re.sub('[^a-zA-Z ]+', '', txt) #only allows for letters
    txt = ' '.join([PorterStemmer().stem(word=word) for word in txt.split(" ") if word not in stop_words ]) # stem & remove stop words
    return txt

def print_model_performance(target,predicted):
#     training_sample = model.predict(X_train)
#     testing_sample = model.predict(X_test)
#     print('training ')
#     print('train accuracy ',accuracy_score(training_sample, y_train))
#     print('\n testing  ')
#     print('test average accuracy ',accuracy_score(testing_sample, y_test))
#     print(confusion_matrix( y_test,testing_sample))
    print('outcome of training')
    print(classification_report( target,predicted))   #uncomment if you want to see full report 
    print('test average accuracy ',accuracy_score( target,predicted))
    print(confusion_matrix( target,predicted))


In [4]:
def sentiment_analyzer_scores(text, threshold=0.05, engl=True):
    analyser = SentimentIntensityAnalyzer()
    if engl:
        trans = text
    else:
        trans = translator.translate(text).text
    score = analyser.polarity_scores(trans)
    lb = score['compound']
    return lb
    if lb >= threshold:
        return 1
    elif (lb > -threshold) and (lb < threshold):
        return 0
    else:
        return -1
def train_test_split_features(train, test,train_feature, target_feature,vectorise):
    y_train = train[target_feature]   
    X_train = train[train_feature]
    y_test = test[target_feature]   
    X_test = test[train_feature]
    feature_names=[]
    if(vectorise):
        vect = TfidfVectorizer(min_df=5, ngram_range=(1, 4)) # create Count vectorizer.
        X_train = vect.fit(X_train).transform(X_train) # transform text_train  into a vector 
        X_test = vect.transform(X_test) 
        feature_names = vect.get_feature_names() # to return all words used in vectorizer
  
    return X_train, X_test, y_train, y_test, feature_names
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
#get this working a bit better later
def all_models():
    #Using the recomended classifiers
    #https://arxiv.org/abs/1708.05070
    GBC = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
    RFC = RandomForestClassifier(n_estimators=500, max_features=0.25, criterion="entropy")
    SVM = SVC(C = 0.01, gamma=0.1, kernel="poly", degree=3, coef0=10.0)
    ETC = ExtraTreesClassifier(n_estimators=1000, max_features="log2", criterion="entropy")
    LR = LogisticRegression(C=1.5,fit_intercept=True)
    # Models that were not included in the paper not from SKlearn
    XGC = XGBClassifier()
    CBC = CatBoostClassifier(silent=True)
    light_gb = lgb.LGBMClassifier()
#     models=[(LR, "linear_regression"),(ETC, "Extra_tree_classifier"),(SVM, "support_vector_classifier"), (RFC, "random_forest_classifier"), (GBC, "gradient_boosted_classifier"),
#              (XGC, "XGBoost"),(light_gb,"Light_GBM"), (CBC, "catboost_classifier")]
    models=[(LR, "linear_regression"),(SVM, "support_vector_classifier"), 
              (CBC, "catboost_classifier"),(XGC, "XGBoost")]
#      (RFC, "random_forest_classifier"),(ETC, "Extra_tree_classifier"),(GBC, "gradient_boosted_classifier"),
#                (CBC, "catboost_classifier")]
    return models

In [5]:
def quick_run(df):
    train, test = train_test_split(df, test_size=0.99)
    train, validation = train_test_split(train, test_size=0.125)
    return train, validation, test

In [ ]:
data_df = pd.read_csv(dataset_location) 

data_df[text_variable] = data_df[text_variable].astype(str)
data_df[text_variable] = data_df[text_variable].apply(clean_text)  
data_df = data_df.dropna()
data_df,_,_=quick_run(data_df)
data_df=data_df.reset_index(drop=True)

data_df

In [ ]:

def run_features(df, model, splits,features=text_variable,vectorise=True):
    cv = KFold(n_splits=splits, random_state=42, shuffle=False)
    full_prediciton=[]
    for train_index, test_index in cv.split(df):
        train, test = df.loc[train_index], df.loc[test_index]

        X_train, X_test, y_train, y_test, feature_names=train_test_split_features(train,test,text_variable,target_feature, True)
#         model = LogisticRegressionCV()
        #to run faster
#         model = RandomForestClassifier()
        model.fit(X_train, y_train)
        #print_model_performance(lgstc, X_train, X_test, y_train, y_test)
        prediction = model.predict(X_test)
        full_prediciton.append(prediction)
    predictions=[]

    for set_of_prediction in full_prediciton:
        for predicted in set_of_prediction:
            predictions.append(predicted)
    return predictions
# to_dataset=data_df.copy()


model_predicted_names=[]
models=all_models()
df_copy=data_df.copy()
for model, name in models:
    print(name)
    import time
    start_time = time.time()
    predictions=run_features(df_copy,model,splits)
    print("--- %s seconds ---" % (time.time() - start_time))
    predicted_name=name+'_prediction'
    df_copy[predicted_name]=predictions
    model_predicted_names.append(predicted_name)
    
    print_model_performance(df_copy[target_feature],predictions)
    predictions.clear()


In [ ]:
def sentiment_analyzer_scores(text, threshold=0.05, engl=True):
    analyser = SentimentIntensityAnalyzer()

    if engl:
        trans = text
    else:
        trans = translator.translate(text).text

    score = analyser.polarity_scores(trans)
    lb = score['compound']
    return lb
    if lb >= threshold:
        return 1
    elif (lb > -threshold) and (lb < threshold):
        return 0
    else:
        return -1

In [ ]:
df_copy['polarity']=df_copy[text_variable].apply(lambda text: TextBlob(text).sentiment.polarity)
df_copy['subjectivity']=df_copy[text_variable].apply(lambda text: TextBlob(text).sentiment.subjectivity)
df_copy['vader_sentiment']=df_copy[text_variable].apply(lambda tweet: sentiment_analyzer_scores(tweet))
model_predicted_names.append('polarity')
model_predicted_names.append('subjectivity')
model_predicted_names.append('vader_sentiment')

In [ ]:
df_copy

In [ ]:
model = RandomForestClassifier()
predictions=run_features(df_copy,model,split, features=model_predicted_names,vectorise=False)
print_model_performance(df_copy[target_feature],predictions)